In [135]:
import requests
from bs4 import BeautifulSoup
import json

In [141]:
url = "https://www.rottentomatoes.com/browse/movies_in_theaters "
response = requests.get(url)
if response.status_code == 200:
    print(response)

In [143]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
print (soup)

<!DOCTYPE html>

<html dir="ltr" lang="en" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/" xmlns="http://www.w3.org/1999/xhtml">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<script charset="UTF-8" crossorigin="anonymous" data-domain-script="7e979733-6841-4fce-9182-515fac69187f" integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN" src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js" type="text/javascript">
</script>
<script type="text/javascript">
                function OptanonWrapper() { }
            </script>
<script ccpa-opt-out-geo="US" ccpa-opt-out-ids="USP" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript">
</script>
<script src="/assets/pizza-pie/javascripts/bundles/roma/rt-common.js?single"></script>
<meta content="text/html; charset=utf-8" http-equiv=

In [133]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
response= soup.title
print (response.get_text())

Rotten Tomatoes: Movies | TV Shows | Movie Trailers | Reviews | Rotten Tomatoes


In [42]:
import requests
from bs4 import BeautifulSoup
def fetch_movies_from_rotten_tomatoes():
    print("🍅 Fetching Latest Movies from Rotten Tomatoes...")
    url = "https://www.rottentomatoes.com/browse/movies_in_theaters"
    headers = {
        "User-Agent": "Mozilla/5.0"  # Mimic a real browser
    }
    try:
        response = requests.get(url, headers=headers)  
        soup = BeautifulSoup(response.content, "html.parser")
        #print(soup)
        titles = []
        for i in soup.select('a.js-tile-link'):
            title = i.get("href").split("/")[-1].replace("-", " ").title()
            titles.append(title)

        print(f"✅ Fetched {len(titles)} movies from Rotten Tomatoes!\n")
        return titles[:30]  # Limit to 30 latest movies

    except requests.exceptions.RequestException as e:
        print(f"❌ Error while scraping Rotten Tomatoes: {e}")
        return []


titles = fetch_movies_from_rotten_tomatoes()
print(titles)

🍅 Fetching Latest Movies from Rotten Tomatoes...
✅ Fetched 11 movies from Rotten Tomatoes!

['Purple_Rain', 'My_Motherland', 'Open_Your_Eyes_Jeffrey', 'Guns_Of_Redemption', 'The_Visitor_2024', 'The_Way_My_Way', 'Play_It_As_It_Lays', 'The_Corpse', 'A_Match', 'You_Burn_Me', 'Ufc_313_Pereira_Vs_Ankalaev']


In [125]:
#1. Title Transformation 
import re
def transform_title(title):
    title = re.sub(r'[^a-zA-Z0-9\s]', '', title)
    title = title.title()
    title = title.strip()
    return title
original_title = "My_Motherland"
transformed_title = transform_title(original_title)

print("Original:", original_title)
print("Transformed:", transformed_title)

Original: My_Motherland
Transformed: Mymotherland


In [127]:
#2.Release Date Transformation 

from datetime import datetime

def transform_release_date(date_str):
    if not date_str or date_str.strip() == "":
        return "Unknown"
    
    try:   
        formatted_date = date_str.replace(" ", "-")
        date_object = datetime.strptime(formatted_date, "%d-%b-%Y")
        transformed_date = date_object.strftime("%Y-%m-%d") 
        return transformed_date
    except ValueError:
        return "Invalid Date Format"

original_date = "25 Jan 2021"
transformed_date = transform_release_date(original_date)

print("Original:", original_date)
print("Transformed:", transformed_date)

missing_date = ""
print("Missing Date Example:", transform_release_date(missing_date))


Original: 25 Jan 2021
Transformed: 2021-01-25
Missing Date Example: Unknown


In [72]:
#3. Genre Transformation 

def transform_genres(genre_str):
    genre_str = genre_str.lower()
    genres = [genre.strip() for genre in genre_str.split(',')]
    seen = set()
    unique_genres = []
    for genre in genres:
        if genre and genre not in seen:
            unique_genres.append(genre)
            seen.add(genre)
    
    return unique_genres
original_genres = "Action, Adventure, Fantasy"
transformed_genres = transform_genres(original_genres)

print("Original:", original_genres)
print("Transformed:", transformed_genres)


Original: Action, Adventure, Fantasy
Transformed: ['action', 'adventure', 'fantasy']


In [84]:
#4. IMDb Rating Transformation 

def transform_imdb_rating(rating_str, normalize=False):
    try:
        rating = float(rating_str)
    except ValueError:
        print(f"Error: Unable to convert '{rating_str}' to a float.")
        return None
    rating = round(rating, 1)
    if normalize:
        rating = round(rating / 10, 1)

    return rating

original_rating = "8.78967549"
transformed_rating = transform_imdb_rating(original_rating, normalize=False)
normalized_rating = transform_imdb_rating(original_rating, normalize=True)

print("Original Rating:", original_rating)
print("Transformed Rating:", transformed_rating)   
print("Normalized Rating:", normalized_rating)      


Original Rating: 8.78967549
Transformed Rating: 8.8
Normalized Rating: 0.9


In [90]:
def transform_actors(actor_string, sort_alphabetically=False):
    actors = actor_string.split(',')
    actors = [actor.strip() for actor in actors]
    if sort_alphabetically:
        actors.sort()
    top_three_actors = actors[:3]
    transformed_string = ', '.join(top_three_actors)
    return transformed_string
original_actors = "  Tom Hanks, Robin Wright, Gary Sinise, Mykelti Williamson  "
transformed_actors = transform_actors(original_actors, sort_alphabetically=True)
print("Transformed:", transformed_actors)

Transformed: Gary Sinise, Mykelti Williamson, Robin Wright


In [94]:
# 6. Box Office Transformation 

def clean_box_office(box_office_str):
    if not box_office_str: 
        return 0
    cleaned_str = re.sub(r'[^0-9]', '', box_office_str)  
    return int(cleaned_str) if cleaned_str else 0  

original = "$1,200,000"
transformed = clean_box_office(original)
print("Original:", original)
print("Transformed:", transformed)

Original: $1,200,000
Transformed: 1200000


In [98]:
def total_awards(text):
    numbers = re.findall(r'\d+', text)
    total = sum(int(num) for num in numbers)
    return total
text = "Won 13 Oscars. Another 15 wins & 10 nominations."
print(total_awards(text))


38


In [106]:
# 8 A. Metascore Transformation 

def transform_metascore(metascore):
    if metascore == "N/A":
        return None  
    try:
        return int(metascore) / 100  
    except ValueError:
        return None  
original_score = "95"
transformed_score = transform_metascore(original_score)
print(f"Original: {original_score}, Transformed: {transformed_score}")

Original: 95, Transformed: 0.95


In [111]:
# 8 B. Metascore Transformation 
def transform_metascore(metascore):
    if metascore == "N/A":
        return None  
    try:
        return int(metascore) / 100  
    except ValueError:
        return None  
original_score = "N/A"
transformed_score = transform_metascore(original_score)
print(f"Original: {original_score}, Transformed: {transformed_score}")

Original: N/A, Transformed: None


In [115]:
# 9.Language Transformation 
def get_movies():
    url = "https://www.rottentomatoes.com/browse/movies_in_theaters/"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to retrieve data")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    movies = [movie.text.strip() for movie in soup.find_all("span", class_="p--small")]
    return movies

def transform_languages(languages):
    if not languages:
        return "Unknown"
    return languages.lower()

movies = get_movies()
language_example = "English, Spanish"
transformed_language = transform_languages(language_example)
print("Original:", language_example)
print("Transformed:", transformed_language)

Original: English, Spanish
Transformed: english, spanish


In [119]:
#10. Production Transformation 
def clean_production_name(name):
    if not name:  
        return "Independent"
    cleaned_name = re.sub(r'[^a-zA-Z0-9 .]', '', name)
    cleaned_name = cleaned_name.replace("Bros.", "Bros")
    return cleaned_name
original_name = "Warner Bros. Pictures!"
transformed_name = clean_production_name(original_name)

print("Original:", original_name)
print("Transformed:", transformed_name)

Original: Warner Bros. Pictures!
Transformed: Warner Bros Pictures


In [145]:
import requests
from bs4 import BeautifulSoup

In [147]:
url = 'https://editorial.rottentomatoes.com/guide/best-horror-movies-of-all-time/'

In [149]:
response=requests.get(url)
print(response)

<Response [200]>


In [151]:
soup= BeautifulSoup(response.content, 'html.parser') 

In [153]:
divs=soup.find_all('div',{'class':'col-sm-18 col-full-xs countdown-item-content'})
print(divs)

[<div class="col-sm-18 col-full-xs countdown-item-content">
<div class="row countdown-item-title-bar">
<div class="col-sm-20 col-full-xs" style="height: 100%;">
<div class="article_movie_title" style="float: left;">
<h2>
<a href="https://www.rottentomatoes.com/m/jaws">Jaws</a>
<span class="subtle start-year">(1975)</span>
<br/>
<img alt="Tomatometer icon" class="icon tiny" decoding="async" src="https://images.fandango.com/cms/assets/ae686540-7fff-11ef-8321-2b978811c524--certified-fresh-notext.svg"> <span class="tMeterScore" style="margin-right: 10px;">97%</span>
</img></h2>
</div>
</div>
<div class="col-sm-4 col-full-xs" style="height: 100%;">
<div class="countdown-index">#1</div>
</div>
</div>
<div class="row countdown-item-details">
<div class="col-sm-24">
<div class="info critics-consensus">
<span class="descriptor">Critics Consensus:</span> Compelling, well-crafted storytelling and a judicious sense of terror ensure Steven Spielberg's <em>Jaws</em> has remained a benchmark in the a

In [155]:
divs[0].find('h2')

<h2>
<a href="https://www.rottentomatoes.com/m/jaws">Jaws</a>
<span class="subtle start-year">(1975)</span>
<br/>
<img alt="Tomatometer icon" class="icon tiny" decoding="async" src="https://images.fandango.com/cms/assets/ae686540-7fff-11ef-8321-2b978811c524--certified-fresh-notext.svg"> <span class="tMeterScore" style="margin-right: 10px;">97%</span>
</img></h2>

In [157]:
headings = [div.find('h2') for div in divs]
print(headings)

[<h2>
<a href="https://www.rottentomatoes.com/m/jaws">Jaws</a>
<span class="subtle start-year">(1975)</span>
<br/>
<img alt="Tomatometer icon" class="icon tiny" decoding="async" src="https://images.fandango.com/cms/assets/ae686540-7fff-11ef-8321-2b978811c524--certified-fresh-notext.svg"> <span class="tMeterScore" style="margin-right: 10px;">97%</span>
</img></h2>, <h2>
<a href="https://www.rottentomatoes.com/m/let_the_right_one_in">Let the Right One In</a>
<span class="subtle start-year">(2008)</span>
<br/>
<img alt="Tomatometer icon" class="icon tiny" decoding="async" src="https://images.fandango.com/cms/assets/ae686540-7fff-11ef-8321-2b978811c524--certified-fresh-notext.svg"> <span class="tMeterScore" style="margin-right: 10px;">98%</span>
</img></h2>, <h2>
<a href="https://www.rottentomatoes.com/m/alien">Alien</a>
<span class="subtle start-year">(1979)</span>
<br/>
<img alt="Tomatometer icon" class="icon tiny" decoding="async" src="https://images.fandango.com/cms/assets/ae686540-7ff

In [159]:
[heading.text for heading in headings]

['\nJaws\n(1975)\n\n 97%\n',
 '\nLet the Right One In\n(2008)\n\n 98%\n',
 '\nAlien\n(1979)\n\n 93%\n',
 '\nGet Out\n(2017)\n\n 98%\n',
 '\nPsycho\n(1960)\n\n 97%\n',
 '\nThe Silence of the Lambs\n(1991)\n\n 95%\n',
 '\nOne Cut of the Dead\n(2017)\n\n 100%\n',
 '\nKing Kong\n(1933)\n\n 97%\n',
 '\nAliens\n(1986)\n\n 94%\n',
 '\nA Quiet Place\n(2018)\n\n 96%\n',
 '\nHalloween\n(1978)\n\n 97%\n',
 '\nWhat We Do in the Shadows\n(2014)\n\n 96%\n',
 '\nDiabolique\n(1955)\n\n 95%\n',
 '\nShaun of the Dead\n(2004)\n\n 92%\n',
 '\nThe Cabinet of Dr. Caligari\n(1919)\n\n 96%\n',
 '\nNosferatu\n(1922)\n\n 97%\n',
 "\nRosemary's Baby\n(1968)\n\n 97%\n",
 '\nThe Wailing\n(2016)\n\n 99%\n',
 '\nBride of Frankenstein\n(1935)\n\n 98%\n',
 '\nEyes Without a Face\n(1960)\n\n 97%\n',
 '\nThe Babadook\n(2014)\n\n 98%\n',
 '\nTrain to Busan\n(2016)\n\n 95%\n',
 '\nRepulsion\n(1965)\n\n 96%\n',
 '\nInvasion of the Body Snatchers\n(1956)\n\n 97%\n',
 '\nLate Night with the Devil\n(2023)\n\n 97%\n',
 "\nThe 